# TFRecord Testing Station

I'm hoping to gain more insight into TFRecord and how to properly use them, 'cause right now, it doesn't seem to really work out that well and I don't know why

In [11]:
import tensorflow as tf

from pathlib import Path


In [12]:
path_to_tfr = Path("/Volumes/BrainMets/Rgb_Brain_Mets/brain_mets_classification/derivatives/TFRecords/patient_data_2classes.tfrecord")

In [9]:
feature_description = {
    "image": tf.io.FixedLenFeature([155, 240, 240, 4], tf.float32), # formerly: [149, 185, 155, 4]
    "sex": tf.io.FixedLenFeature([2], tf.int64, default_value=[0,0]),
    "age": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "primary": tf.io.FixedLenFeature([], tf.int64, default_value=0),
}

data = tf.data.TFRecordDataset([path_to_tfr], compression_type="GZIP")

def parse_record(record):
    return tf.io.parse_single_example(record, feature_description)

data = data.map(parse_record)

data = data.shuffle(buffer_size=1000)

data = data.batch(4)

data = data.prefetch(buffer_size=1)

In [10]:
for record in data.take(1):
    print(record["sex"].numpy())
    print(record["age"].numpy())
    print(record["primary"].numpy())

2024-06-04 12:40:11.397293: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a while): 26 of 1000
2024-06-04 12:40:31.353378: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a while): 76 of 1000
2024-06-04 12:40:41.463648: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a while): 102 of 1000
2024-06-04 12:41:01.113769: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a while): 152 of 1000
2024-06-04 12:41:11.370729: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a while): 177 of 1000
2024-06-04 12:41:31.277364: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a wh

[[0 1]
 [1 0]
 [0 1]
 [1 0]]
[68 63 56 66]
[0 1 0 1]


In [18]:
trainig_fold_paths = [
    "/Users/LennartPhilipp/Desktop/Uni/Prowiss/Data.tmp/3D_CNN_whole/fold_2.tfrecord",
    "/Users/LennartPhilipp/Desktop/Uni/Prowiss/Data.tmp/3D_CNN_whole/fold_3.tfrecord"
]

val_fold_path = "/Users/LennartPhilipp/Desktop/Uni/Prowiss/Data.tmp/3D_CNN_whole/fold_1.tfrecord"

test_fold_path = "/Users/LennartPhilipp/Desktop/Uni/Prowiss/Data.tmp/3D_CNN_whole/fold_0.tfrecord"

feature_description = {
    "image": tf.io.FixedLenFeature([155, 240, 240, 4], tf.float32), # formerly: [149, 185, 155, 4]
    "sex": tf.io.FixedLenFeature([2], tf.int64, default_value=[0,0]),
    "age": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "primary": tf.io.FixedLenFeature([], tf.int64, default_value=0),
}

data = tf.data.TFRecordDataset([trainig_fold_paths], compression_type="GZIP")

def parse_record(record):
    return tf.io.parse_single_example(record, feature_description)

data = data.map(parse_record)

data = data.shuffle(buffer_size=1000)

data = data.batch(10)

data = data.prefetch(buffer_size=1)

for record in data.take(1):
    print(record["sex"].numpy())
    print(record["age"].numpy())
    print(record["primary"].numpy())

[[1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]
[71 30 58 66 25 47]
[1 0 0 0 0 1]


## ChatGPT attempt at k-fold cross validation

In [15]:
tf.random.set_seed(42)

# Define the feature description
feature_description = {
    "image": tf.io.FixedLenFeature([155, 240, 240, 4], tf.float32),
    "sex": tf.io.FixedLenFeature([2], tf.int64, default_value=[0,0]),
    "age": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "primary": tf.io.FixedLenFeature([], tf.int64, default_value=0),
}

# Parse a single record
def parse_record(record):
    return tf.io.parse_single_example(record, feature_description)

# Load the dataset
raw_dataset = tf.data.TFRecordDataset([path_to_tfr], compression_type="GZIP")

# Parse the dataset
parsed_dataset = raw_dataset.map(parse_record)

# Shuffle the dataset (streaming)
shuffled_dataset = parsed_dataset.shuffle(buffer_size=10000)

# Define the number of folds
k = 10
fold_size = 48  # Example fold size; adjust based on your dataset and memory constraints

# Function to write a dataset to a TFRecord file
def write_tfrecord(file_path, dataset):
    writer = tf.io.TFRecordWriter(file_path)
    writer.write(dataset)

# Create and save each fold
for fold_idx in range(k):
    if fold_idx == k - 1:
        fold_dataset = shuffled_dataset.take(fold_size + (len(shuffled_dataset) % fold_size))
    else:
        fold_dataset = shuffled_dataset.take(fold_size)
        shuffled_dataset = shuffled_dataset.skip(fold_size)
    file_path = f"/Users/LennartPhilipp/Desktop/Uni/Prowiss/Data.tmp/3D_CNN_whole/fold_{fold_idx}.tfrecord"
    write_tfrecord(file_path, fold_dataset)


TypeError: write(): incompatible function arguments. The following argument types are supported:
    1. (self: tensorflow.python.lib.io._pywrap_record_io.RecordWriter, record: str) -> None

Invoked with: <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x30fe0bfb0>, <_TakeDataset element_spec={'age': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image': TensorSpec(shape=(155, 240, 240, 4), dtype=tf.float32, name=None), 'primary': TensorSpec(shape=(), dtype=tf.int64, name=None), 'sex': TensorSpec(shape=(2,), dtype=tf.int64, name=None)}>

In [ ]:
import glob

# List of all fold files
fold_files = glob.glob("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Data.tmp/3D_CNN_whole/fold_*.tfrecord")

# Function to create a dataset from a list of TFRecord files
def create_dataset_from_tfrecord(files):
    raw_dataset = tf.data.TFRecordDataset(files, compression_type="GZIP")
    return raw_dataset.map(parse_record)

# Create k-fold datasets
def create_k_fold_datasets(fold_files, k):
    for i in range(k):
        val_files = [fold_files[i]]
        train_files = [f for j, f in enumerate(fold_files) if j != i]
        
        train_dataset = create_dataset_from_tfrecord(train_files)
        val_dataset = create_dataset_from_tfrecord(val_files)
        
        train_dataset = train_dataset.batch(4).prefetch(buffer_size=1)
        val_dataset = val_dataset.batch(4).prefetch(buffer_size=1)
        
        yield train_dataset, val_dataset

# Create the k-fold datasets
k_fold_datasets = create_k_fold_datasets(fold_files, k)


In [ ]:
for fold_idx, (train_dataset, val_dataset) in enumerate(k_fold_datasets):
    print(f"Training fold {fold_idx + 1}/{k}")
    
    # Define your model
    model = create_model()  # Replace with your model creation function
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(train_dataset, epochs=10, validation_data=val_dataset)
    
    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(val_dataset)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")
